In [1]:
import dataAcquisition as da

In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
df=da.getStablePoolHistory(10000000)

In [ ]:
df

In [ ]:
df['returns']=df.feesUSD/df.tvlUSD

In [ ]:
start_date=df[df.pool_name=='USDC_USDT_01'].date.min()

In [ ]:
current_env=df[df.date>start_date].reset_index(drop=True)

In [ ]:
current_env.groupby('pool_name').returns.mean()*365

In [ ]:
current_env.groupby('pool_name').agg({'returns':['mean','std']})

In [ ]:
def returnsColumn(gb_tupple):
    pool_name,frame=gb_tupple
    series=frame.sort_values('date',ascending=True).returns.reset_index(drop=True)
    series.name=pool_name
    return series

In [ ]:
pd.concat([returnsColumn(tupple) for tupple in current_env.groupby('pool_name')],axis=1)

In [ ]:
busd=current_env[current_env.pool_name=='BUSD_USDC_05'].copy()

In [ ]:
busd_range=pd.date_range(busd.date.min(),busd.date.max())
len(busd_range[~busd_range.isin(busd.date)])

In [ ]:
busd_missing=busd_range[~busd_range.isin(busd.date)].to_frame().reset_index(drop=True)
busd_missing.columns=['date']
busd_missing['pool_name']='BUSD_USDC_05'
busd_missing['returns']=0
current_env_clean=pd.concat([current_env[['date','pool_name','returns']],busd_missing]).reset_index(drop=True)

In [ ]:
returns_df=pd.concat([returnsColumn(tupple) for tupple in current_env_clean.groupby('pool_name')],axis=1)

In [ ]:
returns_df

In [ ]:
returns_df.corr()

In [ ]:
import weight_simulation as ws

In [ ]:
results=ws.run(returns_df,100000)

In [ ]:
def annualize(results_df):
    results_df['annual_returns']=results_df.daily_return*365
    results_df['annual_std']=results_df.daily_std*365**(1/2)

In [ ]:
annualize(results)

In [ ]:
identity=np.matrix(np.identity(5))

In [ ]:
identity_matrix=np.matrix(np.identity(5))
base_case=run(returns_df,identity_matrix)

In [ ]:
annualize(base_case)

In [ ]:
plt.scatter(results.annual_std,results.annual_returns,color='lightgrey')
for index, row in base_case.iterrows():
    pool_name=base_case.columns[index]
    plt.scatter([row.annual_std],[row.annual_returns],label=pool_name)
_=plt.legend()

In [ ]:
base_case